In [159]:
import numpy as np
import pandas as pd
import os
from checkRF_Score import *
from codes.utils.file import *
from codes.parameters import ASTRAL, WQFM, TRUE_ST, ASTEROID_RAxML_NG, ASTRID_RAxML_NG, ASTRALMP_RAxML_NG, ASTRID_FASTME_RAxML_NG, \
    BIPSTEP_RAxML_NG, FASTRFS_RAxML_NG_GREEDY, FASTRFS_RAxML_NG_SINGLE, METHOD_TO_ST

In [160]:
dataset_path = join_dir("dataset", "asteroid_dataset", "withils")
# dataset_path = join_dir("dataset","asteroid_simulated", "asteroid_data","simulations", "withoutils")

In [161]:
# in asteroid dataset
gt_file_name="true.true.geneTree.newick"
# gt_file_name = "raxml-ng.GTR+G.geneTree.newick"
species_tree_name="speciesTree.newick"
# species_tree_name= "fastrfs-raxml-ng_greedy.GTR+G.speciesTree.newick"

In [162]:
# for our experiment
gt_output_folder= "output"
gt_output_file_name= "1_gt.tre"
truncated_gt_file_name= "1_gt_truncated.tre"

true_species_tree_name= "true-species.out.tree"
RF_SCORES_OUTPUT_DIR = "stats/RF_scores"
OUTPUT_ST_NAME = "method_st.newick"

In [163]:
def remove_and_create_output_folder(dir_current_path):
    remove_dir_current(dir_current_path)
    create_dir_current(dir_current_path)

In [164]:
import re

def remove_one(input_str):
    output_str = re.sub(r"1(?=[^\d_])", "", input_str)
    return output_str

def remove_numbers_after_colon(input_str):
    output_str = re.sub(r':[^,\)]+', '', input_str)
    output_str = remove_one(output_str)
    return output_str

def remove_numbers_before_and_after_colon(input_str):
    output_str = re.sub(r'((\d+\.\d+)|\d+):[^,\)]+', '', input_str)
    return output_str

def remove_numbers_after_colon_NO_NEED_TO_REMOVE_ONE(input_str):
    output_str = re.sub(r':[^,\)]+', '', input_str)
    return output_str

# like remove 0.42, 0.92 
def remove_numbers_after_decimal(input_str):
    output_str = re.sub(r'0\.\d+', '', input_str)
    return output_str

In [165]:
# provide families folder path
def get_all_GTs(families_folder):
    all_gts = []
    for dir in os.listdir(families_folder):
        # print(dir)
        # save mapping from mapping folder
        mapping={}
        mapping_dir="mappings"
        mapping_file_path=families_folder+"/"+dir+"/"+mapping_dir+"/treerecs_mapping.link"
        if os.path.exists(mapping_file_path):
            # print(mapping_file_path)
            with open(mapping_file_path) as f:
                for line in f:
                    # print(line)
                    (key, val) = line.split()
                    mapping[key] = val
        else:
            print("mapping file not found")
        if len(mapping) == 0:
            print("mapping is empty")
        # print(mapping)
        gt_tree_dir="gene_trees"
        gt_tree_file_path=families_folder+"/"+dir+"/"+gt_tree_dir+"/"+gt_file_name
        if not os.path.exists(gt_tree_file_path):
            print("file not exists")
            exit(1)
        with open(gt_tree_file_path) as f:
            line = f.readline()
            line = remove_numbers_after_colon(line)
            # print(line)
            for key in mapping:
                line = line.replace(key, mapping[key])
            all_gts.append(line)
            # print(line)
    #output to a file in output folder
    number_of_gene_trees = len(all_gts)
    with open(gt_output_folder+"/"+gt_output_file_name, "w") as f:
        for gt in all_gts:
            f.write(gt+"\n")
            # f.write(gt)
    return number_of_gene_trees

In [166]:
def get_True_Species_Tree(source_folder):
    # copy species tree to output folder
    # species_tree_path = source_folder+"/species_trees/"+species_tree_name
    species_tree_path = join_dir(source_folder, "species_trees", species_tree_name)
    with open(species_tree_path) as f:
        line = f.readline()
        line = remove_numbers_after_colon_NO_NEED_TO_REMOVE_ONE(line)
        with open(gt_output_folder+"/"+true_species_tree_name, "w") as f:
            f.write(line)


In [167]:
def get_Species_Tree(source_folder, tree_name, method_name):
    species_tree_path = join_dir(source_folder, "species_trees", tree_name)
    with open(species_tree_path) as f:
        line = f.readline()
        if method_name == ASTRALMP_RAxML_NG:
            line = remove_numbers_before_and_after_colon(line)
        line = remove_numbers_after_colon_NO_NEED_TO_REMOVE_ONE(line)
        with open(gt_output_folder+"/"+OUTPUT_ST_NAME, "w") as f:
            f.write(line)
    f.close()

    copy_file_current(f"{gt_output_folder}/{OUTPUT_ST_NAME}", f"{gt_output_folder}/input.nwk")

In [168]:
def getTaxaSet(t):
    return t.replace('(', '').replace(')', '').split(';')[0].split(':')[0].split(',')

In [169]:
def get_INPUT_DATA():
    OUTGROUP = 'GAL'
    st = open('output/input.nwk')
    st = st.read()
    gts = open(f'output/{truncated_gt_file_name}')

    gts = gts.read().split(';\n')[:-1] 

    # print(len(gts))

    # print(st)
    taxaSet = getTaxaSet(st)
    # print(taxaSet)
    matrix = np.zeros((len(taxaSet), len(gts)), dtype=int)


    # print(len(gts))
    for gt_idx, gt in  enumerate(gts) :
        # print(gt)
        cur_taxaSet = getTaxaSet(gt)
        # print(cur_taxaSet)
        for i, tx in enumerate(taxaSet):
            if tx  in cur_taxaSet or tx == OUTGROUP:
                matrix[i][gt_idx] = 1
    with open('output/input.data', 'w', newline='') as f:
        print(matrix.shape[0], matrix.shape[1], file=f)
        for i in range(len(taxaSet)):
            # f.write(matrix[i].tostring(), taxaSet[i])
            print(*matrix[i],taxaSet[i], end='\n', file=f)

In [170]:
def getTerraceSize():
    # !rm -rf terrace_output
    remove_dir_current("terrace_output")
    # !mkdir -p terrace_output
    create_dir_current("terrace_output")
    
    !./bin/raxml-ng output/input.nwk output/input.data "size"> terrace_output/terrace_output.txt

In [171]:
def getConsensusTree():
    # get consensus tree
    terrace_dir = join_dir(TERRACE_OUTPUT_FOLDER)
    consensus_dir = join_dir(terrace_dir, "consensus")
    !rm -rf "$consensus_dir"
    !mkdir "$consensus_dir"

    if not dir_exists_abs(terrace_dir):
        print("Directory does not exist: {}".format(terrace_dir))
        exit(1)
    for terrace_file in os.listdir(terrace_dir):
        file_path = join_dir(terrace_dir, terrace_file)
        if file_path.endswith(".txt"):
            # print("Processing file: {}".format(file_path))
            terrace_file_name = terrace_file.split(".")[0]
            !./bin/raxml-ng --consense MRE --tree "$file_path" --prefix "$consensus_dir"/"$terrace_file_name"

In [172]:
def get_ASTRAL_tree():
    # !rm -rf in/
    remove_dir_current("in")
    # !rm -rf out/
    remove_dir_current("out")
    # !mkdir in
    create_dir_current("in")
    # !cp output/1_gt.tre in/
    copy_file_current(f"output/{truncated_gt_file_name}", "in/1_gt.tre")
    
    !python3 postprocessor.py
    # !cp out/complete/astral/1_gt_species.out.tre output/input.nwk
    copy_file_current("out/complete/astral/1_gt_species.out.tre", f"{gt_output_folder}/input.nwk")

In [173]:
def get_True_ST_as_speciesTree():
    copy_file_current(f"{gt_output_folder}/{true_species_tree_name}", f"{gt_output_folder}/input.nwk")

In [174]:
# def get_asteroid_raxml_as_ST():
    # copy_file_current

In [175]:
def get_wQFM_tree():
    # !rm -rf in/
    remove_dir_current("in")
    # !rm -rf out/
    remove_dir_current("out")
    # !mkdir in
    create_dir_current("in")
    # !cp output/1_gt.tre in/
    copy_file_current(f"output/{truncated_gt_file_name}", "in/1_gt.tre")
    
    !python3 postprocessor.py
    # !cp out/complete/astral/1_gt_species.out.tre output/input.nwk
    copy_file_current("out/complete/wQFM/1_gt_species.tre", "output/input.nwk")

In [176]:
def get_ASTRAL_RF_Score(true_tree):
    astral_rf_score = calculate_RF_distance_for_a_ST(join_dir(gt_output_folder, "input.nwk"), true_tree)
    print("astral_rf_score --> ", astral_rf_score)
    return astral_rf_score

In [177]:
def get_Terrace_RF_Scores(true_tree):
    
    #get RF score for each tree in Terrace
    terrace_rf_scores=calculate_RF_distance_for_terrace(join_dir(TERRACE_OUTPUT_FOLDER, "terrace_output.txt"), true_tree)
    # print("terrace_rf_scores", terrace_rf_scores)
    if terrace_rf_scores is not None:
        return terrace_rf_scores
    else:
        return []

In [178]:
def get_Consensus_RF_score(true_tree):
    terrace_dir = join_dir(TERRACE_OUTPUT_FOLDER)
    consensus_dir = join_dir(terrace_dir, "consensus")
    CONSENSUS_TREE_EXTENSION = "consensusTreeMRE"
    file_path = join_dir(consensus_dir, "terrace_output.raxml.consensusTreeMRE")
    # print("Processing file: {}".format(file_path))
    # print("True Tree --> ", true_tree)
    # remove_numbers_from_consensus_tree(file_path)
    if file_exists_absolute(file_path):
        dist = calculate_RF_distance_for_a_ST(file_path, true_tree)
        print("Consensus RF distance: {}".format(dist))
        return dist
    else:
        print("File does not exist: {}".format(file_path))
        return -1

In [179]:
def Keep_few_STs_in_Terrace(KEEPING_STs_NUM):
    num_of_trees_in_terrace = 0
    for file in os.listdir("terrace_output"):
        if file.endswith(".txt"):
            file_path = "terrace_output/"+file
            
            if os.path.getsize(file_path) <= 0:
                return -1 # no trees in terrace
            
            df = pd.read_csv(file_path)
            num_of_trees_in_terrace = len(df)
            if len(df) > KEEPING_STs_NUM:
                df = df.iloc[:KEEPING_STs_NUM]
                df.to_csv(file_path, index=False)
    return num_of_trees_in_terrace

In [180]:
def get_stats_of_terrace(terrace_rf_scores,true_tree, astral_rf):
    return calculate_RF_AND_generate_stats_OF_TERRACE_Return_better_trees(terrace_rf_scores,true_tree, astral_rf)

In [181]:
import datetime
def get_time():
    now = datetime.datetime.now()
    return now.strftime("%d_%b_%Y_%H_%M%p")

In [182]:
def keep_first_few_lines(input_file_path, output_file_path, num_lines_to_keep):
    with open(input_file_path, 'r') as input_file:
        lines = input_file.readlines()
    
    if len(lines) < num_lines_to_keep:
        print("WARNING: File {} has only {} lines, but {} lines were requested to be kept. Keeping all lines.".format(input_file_path, len(lines), num_lines_to_keep))
        num_lines_to_keep = len(lines)

    filtered_lines = lines[:num_lines_to_keep]  # Keep only the first num_lines_to_keep lines

    with open(output_file_path, 'w') as output_file:
        output_file.writelines(filtered_lines)

In [183]:
def getSizeFromFile(terrace_size_file):
    with open(terrace_size_file, 'r') as terrace_size_file:
        lines = terrace_size_file.readlines()
        return int(lines[0])    

In [184]:
METHODS_FROM_ASTEROID_DATASET = [ASTEROID_RAxML_NG, ASTRID_RAxML_NG, ASTRALMP_RAxML_NG, ASTRID_FASTME_RAxML_NG, BIPSTEP_RAxML_NG, FASTRFS_RAxML_NG_GREEDY, FASTRFS_RAxML_NG_SINGLE]

def run_analysis(numberOfSpecies, USED_METHOD=ASTRAL):   
    csv_out = []
    csv_out.append(["Model Condition", "Number of Species", "Number of Gene Trees", "Terrace size of "+USED_METHOD])
    i=0
    MODEL_CONDITION = numberOfSpecies

    #remove and create dir of rf scores
    MODEL_CONDITION_DIR = f"{RF_SCORES_OUTPUT_DIR}/{MODEL_CONDITION}"
    remove_and_create_output_folder(MODEL_CONDITION_DIR)

    for model_condition in sorted(os.listdir(dataset_path)):
        if MODEL_CONDITION not in model_condition:
            continue
        
        i+=1
        # skip_list = [8, 10, 14]
        # if i in skip_list:
        #     continue

        print("Model Condition --> ", model_condition)

        csv_out.append([model_condition, numberOfSpecies, "", ""])

        remove_and_create_output_folder(gt_output_folder) 

        source_folder = join_dir(dataset_path, model_condition)
        # print(source_folder)
        families_folder = join_dir(source_folder, "families")
        
        number_of_gene_trees = get_all_GTs(families_folder)
        
        get_True_Species_Tree(source_folder)
        
        true_tree = join_dir(gt_output_folder, true_species_tree_name)

        all_gts_path = join_dir(gt_output_folder, gt_output_file_name)
        truncated_gt_path = join_dir(gt_output_folder, truncated_gt_file_name)
        
        number_gene_trees_arr = [ i for i in range(100, number_of_gene_trees+1, 100) ]
        if number_of_gene_trees not in number_gene_trees_arr:
            number_gene_trees_arr.append(number_of_gene_trees)    
        print(number_of_gene_trees)
        print("number_gene_trees_arr --> ", number_gene_trees_arr)

        for NUMBER_OF_GT in number_gene_trees_arr:
            print("NUMBER_OF_GT --> ", NUMBER_OF_GT)
            keep_first_few_lines(all_gts_path, truncated_gt_path, NUMBER_OF_GT)
            
            if USED_METHOD == ASTRAL:
                get_ASTRAL_tree()
            elif USED_METHOD == WQFM:
                get_wQFM_tree()
            elif USED_METHOD == TRUE_ST:
                get_True_ST_as_speciesTree()
            elif USED_METHOD in METHODS_FROM_ASTEROID_DATASET:
                get_Species_Tree(source_folder, METHOD_TO_ST[USED_METHOD], USED_METHOD)
            else:
                print("Invalid method")
                return 

            get_INPUT_DATA()

            # print("GET TERRACE")
            getTerraceSize()
            # print("GET TERRACE DONE")
            terrace_size_file = join_dir("terrace_size.txt")
            number_of_trees_in_terrace = getSizeFromFile(terrace_size_file)
            print("number_of_trees_in_terrace --> ", number_of_trees_in_terrace)
            
            if number_of_trees_in_terrace <= 0:
                print("\n\n=============No trees in terrace================\n\n")
                # continue
            
            csv_out.append(["", "", NUMBER_OF_GT, number_of_trees_in_terrace])
        
        csv_out.append(["", "", "", ""])
        
        if i>=20:
            break
    
    stat_fl = f'stats/Summary_{MODEL_CONDITION}_{get_time()}_{USED_METHOD}.csv'
    with open(stat_fl, 'w') as stat_file:
        csv.writer(stat_file).writerows(csv_out)

In [185]:

# MODEL_CONDITIONS = ["s25", "s50", "s75", "s100"]
MODEL_CONDITIONS = [ "s50" ]
for model_condition in MODEL_CONDITIONS:
    print(f"\n\nstarting analysis for {model_condition} at {get_time()}\n\n")
    run_analysis(model_condition, USED_METHOD=TRUE_ST)
    print(f"\n\nfinished analysis for {model_condition} at {get_time()}\n\n")



starting analysis for s50 at 21_May_2023_01_23AM


Model Condition -->  ssim_veryhighmiss_s50_f1000_sites100_GTR_bl0.05_d0.0_l0.0_t0.0_gc0.0_p0.0_pop50000000_ms0.6_mf0.6_seed3000
835
number_gene_trees_arr -->  [100, 200, 300, 400, 500, 600, 700, 800, 835]
NUMBER_OF_GT -->  100
number_of_trees_in_terrace -->  2095797375
NUMBER_OF_GT -->  200
number_of_trees_in_terrace -->  3070305
NUMBER_OF_GT -->  300
number_of_trees_in_terrace -->  8415
NUMBER_OF_GT -->  400
number_of_trees_in_terrace -->  135
NUMBER_OF_GT -->  500
number_of_trees_in_terrace -->  135
NUMBER_OF_GT -->  600
number_of_trees_in_terrace -->  15
NUMBER_OF_GT -->  700
number_of_trees_in_terrace -->  5
NUMBER_OF_GT -->  800
number_of_trees_in_terrace -->  5
NUMBER_OF_GT -->  835
number_of_trees_in_terrace -->  5
Model Condition -->  ssim_veryhighmiss_s50_f1000_sites100_GTR_bl0.05_d0.0_l0.0_t0.0_gc0.0_p0.0_pop50000000_ms0.6_mf0.6_seed3001
778
number_gene_trees_arr -->  [100, 200, 300, 400, 500, 600, 700, 778]
NUMBER_OF_GT --